In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn import Transformer
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.metrics import roc_curve, roc_auc_score
from tqdm import tqdm
from transformers import BertModel, BertTokenizer, BertConfig, BertForSequenceClassification
import pandas as pd
import numpy as np
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

In [15]:
#pd.set_option('display.max_colwidth', None)  # 열에 출력되는 최대 문자열 길이 제한을 해제

In [3]:
#데이터 파일 로드
data_6h = pd.read_csv('/data2/project/2024summer/jhy0812/prodata4(icd_10)/data_6h.csv')
data_12h = pd.read_csv('/data2/project/2024summer/jhy0812/prodata4(icd_10)/data_12h.csv')
data_24h = pd.read_csv('/data2/project/2024summer/jhy0812/prodata4(icd_10)/data_24h.csv')
data_48h = pd.read_csv('/data2/project/2024summer/jhy0812/prodata4(icd_10)/data_48h.csv')

label = pd.read_csv('/data2/project/2024summer/jhy0812/prodata4(icd_10)/label.csv')

In [4]:
import pandas as pd

def process_all_subjects(data_df, label_df, interval_hours):
    results = []

    # 'time' 열을 datetime 형식으로 변환
    data_df['time'] = pd.to_datetime(data_df['time'])

    # 모든 라벨 데이터를 순회
    for _, label_row in label_df.iterrows():
        subject_id = label_row['subject_id']
        discharge_date = pd.to_datetime(label_row['discharge'])

        # 간격에 따라 시간 포인트 생성
        if interval_hours == 3:
            start_time = discharge_date - timedelta(days=1) - pd.Timedelta(hours=3)
            time_points = [start_time - pd.Timedelta(hours=i * 3) for i in range(10)]
        elif interval_hours == 6:
            start_time = discharge_date - timedelta(days=1) - pd.Timedelta(hours=6)
            time_points = [start_time - pd.Timedelta(hours=i * 6) for i in range(10)]
        elif interval_hours == 12:
            start_time = discharge_date - timedelta(days=1) - pd.Timedelta(hours=12)
            time_points = [start_time - pd.Timedelta(hours=i * 12) for i in range(10)]
        elif interval_hours == 24:
            start_time = discharge_date - timedelta(days=1) - pd.Timedelta(hours=24)
            time_points = [start_time - pd.Timedelta(hours=i * 24) for i in range(10)]
        elif interval_hours == 48:
            start_time = discharge_date - timedelta(days=1) - pd.Timedelta(hours=48)
            time_points = [start_time - pd.Timedelta(hours=i * 48) for i in range(10)]

        # 데이터에서 subject_id의 데이터를 필터링
        subject_data = data_df[data_df['subject_id'] == subject_id]

        # 시간과 대응되는 데이터 쌍을 채워 넣기
        time_item_pairs = []
        for time_point in reversed(time_points):  # 가장 최근 시간부터가 아니라 가장 일찍 나온 시간부터 넣도록 역순으로 변경
            match = subject_data[subject_data['time'] == time_point]
            if not match.empty:
                row = match.iloc[0]
                itemids = row['itemids']
                if isinstance(itemids, str):
                    itemids = eval(itemids)  # 문자열로 저장된 리스트를 리스트로 변환
                itemid_str = ','.join(map(str, itemids))  # 리스트를 문자열로 변환하고 ','로 연결
                entry = itemid_str
                time_item_pairs.append(entry)
            else:
                time_item_pairs.append("")  # 데이터가 없는 경우 빈 값으로 채우기

        combined_time_items = "|".join(time_item_pairs)

        # 결과를 딕셔너리로 저장
        final_result = {
            'subject_id': subject_id,
            'discharge': discharge_date,
            'LABEL_DEAD1_ALIVE0': label_row['LABEL_DEAD1_ALIVE0'],
            f'{interval_hours}H': combined_time_items
        }

        results.append(final_result)

    # 결과 리스트를 데이터프레임으로 변환
    final_results_df = pd.DataFrame(results)
    return final_results_df


In [5]:
#final_result_3h = process_all_subjects(data_3h, label, interval_hours=3)

final_result_6h = process_all_subjects(data_6h, label, interval_hours=6)

final_result_12h = process_all_subjects(data_12h, label, interval_hours=12)

final_result_24h = process_all_subjects(data_24h, label, interval_hours=24)

final_result_48h = process_all_subjects(data_48h, label, interval_hours=48)

In [6]:
#final_result_3h.to_csv('/data2/project/2024summer/jhy0812/prodata3/data_label_3h.csv', index = False)
final_result_6h.to_csv('/data2/project/2024summer/jhy0812/prodata4(icd_10)/data_label_6h.csv', index = False)
final_result_12h.to_csv('/data2/project/2024summer/jhy0812/prodata4(icd_10)/data_label_12h.csv', index = False)
final_result_24h.to_csv('/data2/project/2024summer/jhy0812/prodata4(icd_10)/data_label_24h.csv', index = False)
final_result_48h.to_csv('/data2/project/2024summer/jhy0812/prodata4(icd_10)/data_label_48h.csv', index = False)

# Merge

In [7]:
# 우선 'subject_id'와 'discharge'를 기준으로 데이터를 병합합니다.
merged_df = final_result_6h.merge(final_result_12h[['subject_id', 'discharge', '12H']],
                                  on=['subject_id', 'discharge'], how='left')

merged_df = merged_df.merge(final_result_24h[['subject_id', 'discharge', '24H']],
                                  on=['subject_id', 'discharge'], how='left')

merged_df = merged_df.merge(final_result_48h[['subject_id', 'discharge', '48H']],
                            on=['subject_id', 'discharge'], how='left')

In [12]:
# 우선 'subject_id'와 'discharge'를 기준으로 데이터를 병합합니다.
merged_df = final_result_3h.merge(final_result_6h[['subject_id', 'discharge', '6H']],
                                  on=['subject_id', 'discharge'], how='left')

merged_df = merged_df.merge(final_result_12h[['subject_id', 'discharge', '12H']],
                                  on=['subject_id', 'discharge'], how='left')

merged_df = merged_df.merge(final_result_24h[['subject_id', 'discharge', '24H']],
                            on=['subject_id', 'discharge'], how='left')

merged_df = merged_df.merge(final_result_48h[['subject_id', 'discharge', '48H']],
                            on=['subject_id', 'discharge'], how='left')

In [8]:
merged_df.to_csv('/data2/project/2024summer/jhy0812/prodata4(icd_10)/data_label_merge.csv', index = False)

In [9]:
# 데이터를 병합한 후 TSV 형식으로 저장
merged_df.to_csv('/data2/project/2024summer/jhy0812/prodata4(icd_10)/data_label_merge.tsv', sep='\t',index=False)

print("Data successfully saved as merged_data.tsv")


Data successfully saved as merged_data.tsv
